# Read test radar test file and define gridding and interpolation scheme

In [ ]:
import xarray as xr
from open_radar_data import DATASETS
import xradar as xd
import matplotlib.pyplot as plt
import numpy as np

import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))
import src.global_variables as global_vars
import src.local_variables as local_vars

**Read the radar data**

In [ ]:
data_dir = local_vars.radar_file_dir
# filename = data_dir + '/2024/06/01/rainbow5/HUR/240km_12ele_DP_PROD011.vol/2024060112000700dBZ.vol'
filename = "../data/01-06-2024/2024060112000700dBZ.vol"

In [ ]:
def load_all_elevations(filename, parameter="DBZH"):
    """Load all 12 sweeps for one parameter"""
    elevations = {}
    for i in range(12):
        ds = xr.open_dataset(filename, group=f"sweep_{i}", engine="rainbow")
        elevations[i] = ds
    return elevations

data = load_all_elevations(filename)

**Plot lowest elevation**

In [ ]:
range_vals = data[0].range.values  # shape (480,)
azimuth_vals = data[0].azimuth.values  # shape (360,) 
dbzh_vals = data[0].DBZH.values  # shape (360, 480)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Manual plot with imshow
im1 = ax1.imshow(dbzh_vals, 
                 extent=[range_vals.min(), range_vals.max(), 
                        azimuth_vals.min(), azimuth_vals.max()],
                 aspect='auto', 
                 origin='lower',
                 cmap='viridis')
ax1.set_xlabel('Range (m)')
ax1.set_ylabel('Azimuth (degrees)')
ax1.set_title('Manual imshow plot')
plt.colorbar(im1, ax=ax1, label='DBZH (dBZ)')

# Built-in xarray plot for comparison
data[0].DBZH.plot(ax=ax2, cmap='viridis')
ax2.set_title('Built-in xarray plot')

plt.tight_layout()
plt.show()

**Convert from polar to Cartesian coordinates**

**Import grid**

In [ ]:
grid = np.load("../data/radar_hurum_grid_10x10_8km_spacing.npz")
x_grid = grid['x_bounds_m']
y_grid = grid['y_bounds_m']
z_grid = grid['z_levels_m']

In [ ]:
rd = rd.xradar.georeference()
rd.DBZH.plot(x="x", y="y")

In [ ]:
rd.range.values

In [ ]:
def pol2cart(rho, phi):
   y = rho * np.cos(phi)
   x = rho * np.sin(phi)
   return x, y

def to_cartesian(ds, resolution=1000):

   rangestep = ds.range.head(2).diff(dim='range').values[0]
   xi = np.arange(-ds.range.max() - rangestep/2, ds.range.max() + rangestep/2, resolution)
   yi = np.arange(-ds.range.max() - rangestep/2, ds.range.max() + rangestep/2, resolution)
   [XI, YI] = np.meshgrid(xi, yi)

   [R, PHI] = np.meshgrid(ds.range, ds.azimuth*np.pi/180)
   [X, Y] = pol2cart(R, PHI)

   ds['x'] = xi
   ds['y'] = yi
   ds.x.attrs = {"standard_name": "projection_x_coordinate", "units": "m"}
   ds.y.attrs = {"standard_name": "projection_y_coordinate", "units": "m"}

   def griddata(da, dr, dazimuth):
       zi = scipy.interpolate.griddata(np.c_[X.flatten(), Y.flatten()],
                                       da.flatten(),
                                       np.c_[XI.flatten(), YI.flatten()],
                                       method='nearest')
       zi = zi.reshape(len(xi), len(yi))
       return zi

   zi = xr.apply_ufunc(griddata, ds, ds.range, ds.azimuth,
                       input_core_dims=[['azimuth','range'],['range'],['azimuth']],
                       output_core_dims=[['x','y']],
                       on_missing_core_dim='drop',
                       keep_attrs=True)

   #lon, lat = pyproj.Proj.from_crs(ds.crs_wkt.attrs['crs_wkt'], 'epsg:4326', always_xy=True).transform(*np.meshgrid(ds.x, ds.y))
   for var in zi.data_vars:
       zi[var].attrs['grid_mapping'] = 'crs_wkt'

   zi = zi.where((zi.x**2 + zi.y**2) < ds.range.max()**2)
   zi['time'] = ds.time.min()
   zi = zi.set_coords(['time'])
   zi = zi.expand_dims(dim='time', axis=0)
   zi.encoding["unlimited_dims"] = 'time'
   zi = zi.drop_vars(zi.coords.keys() ^ ('x','y','time','latitude','longitude','altitude','crs_wkt'))
   return zi